In [1]:
import torch
import numpy as np
from torch_geometric.datasets import AmazonBook
import pandas as pd

In [2]:
## importing in the train and test sets
users_df = pd.read_csv('user_list.csv')
entity_df = pd.read_csv('entity_list.csv')
item_df = pd.read_csv('item_list.csv')
categories_df = pd.read_csv('category_list.csv')


## need to get the filtered items list as well

taobao_df = pd.read_csv("taobao/raw/UserBehavior.csv", header = None)
print(taobao_df.head())
taobao_df.columns = ['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp']
taobao_df.head()


   0        1        2   3           4
0  1  2268318  2520377  pv  1511544070
1  1  2333346  2520771  pv  1511561733
2  1  2576651   149192  pv  1511572885
3  1  3830808  4181361  pv  1511593493
4  1  4365585  2520377  pv  1511596146


,user_id,item_id,category_id,behavior_type,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [3]:
print(users_df.head())
print(entity_df.head())
print(item_df.head())
print(categories_df.head())

   usermap_id  user_id
0           0    10502
1           1   710733
2           2   412015
3           3   309344
4           4    49306
   map_id    og_id  type
0       0  4910621  item
1       1  4847970  item
2       2  2736814  item
3       3  2247338  item
4       4  3840111  item
   itemmap_id  item_id
0           0  4910621
1           1  4847970
2           2  2736814
3           3  2247338
4           4  3840111
   categorymap_id  category_id
0         4162024      1003418
1         4162025      1047470
2         4162026      2819583
3         4162027      1363318
4         4162028      2560150


In [20]:
## choosing the top 25000 items with the most interactions
taobao_df = taobao_df[taobao_df['behavior_type'].isin(['cart', 'buy'])]
top_items = taobao_df.groupby('item_id')['user_id'].nunique().reset_index()
top_items = top_items.sort_values('user_id', ascending = False)
top_items.head()

,item_id,user_id
829125,3031354,2355
700290,2560262,1705
853847,3122135,1646
637576,2331370,1588
221765,812879,1558


In [22]:
top_items['rank'] = top_items['user_id'].rank(ascending = False)
top_items.head()
top_items_25 = top_items[top_items['rank']< 5000]


In [23]:
top_items_25.head()
print(top_items_25.shape)
print(top_items_25.head())

(5025, 3)
        item_id  user_id  rank
829125  3031354     2355   1.0
700290  2560262     1705   2.0
853847  3122135     1646   3.0
637576  2331370     1588   4.0
221765   812879     1558   5.0


In [24]:
print(item_df.head())

   itemmap_id  item_id
0           0  4910621
1           1  4847970
2           2  2736814
3           3  2247338
4           4  3840111


In [25]:
## selecting 25000 items : 

item_df_filtered = item_df[item_df['item_id'].isin(top_items_25['item_id'])]
item_df_filtered.head()

,itemmap_id,item_id
4090,4090,3840220
4490,4490,5004166
5451,5451,3245292
5571,5571,3826932
6766,6766,1358635


In [26]:
item_df_filtered.shape

(5025, 2)

In [27]:
item_df_filtered = item_df_filtered.reset_index()
item_df_filtered.head()

,index,itemmap_id,item_id
0,4090,4090,3840220
1,4490,4490,5004166
2,5451,5451,3245292
3,5571,5571,3826932
4,6766,6766,1358635


In [28]:
item_df_filtered = item_df_filtered.drop(columns = 'index')
item_df_filtered = item_df_filtered.reset_index()
item_df_filtered.head()

,index,itemmap_id,item_id
0,0,4090,3840220
1,1,4490,5004166
2,2,5451,3245292
3,3,5571,3826932
4,4,6766,1358635


In [29]:
item_df_filtered = item_df_filtered.rename(columns = {'index': 'new_itemmap_id'})

In [30]:
item_df_filtered.head()

,new_itemmap_id,itemmap_id,item_id
0,0,4090,3840220
1,1,4490,5004166
2,2,5451,3245292
3,3,5571,3826932
4,4,6766,1358635


In [31]:
item_df_filtered.shape
print(len(item_df_filtered['new_itemmap_id'].unique()))

5025


In [50]:
item_df_filtered.to_csv('item_list_filtered2.csv', index = False)

In [55]:
test = pd.read_csv('item_list_filtered2.csv')
test.head()

,new_itemmap_id,itemmap_id,item_id
0,0,4090,3840220
1,1,4490,5004166
2,2,5451,3245292
3,3,5571,3826932
4,4,6766,1358635


In [56]:
test.shape

(5025, 3)

In [35]:
## mapping to the rest of the dataset now
taobao_filtered = taobao_df[taobao_df['item_id'].isin(item_df_filtered['item_id'])]
print(taobao_filtered.shape)
print(taobao_filtered.head())

(1087448, 5)
     user_id  item_id  category_id behavior_type   timestamp
126      100  4840649      1029459           buy  1511790130
398  1000011  3991727      3108044           buy  1511572004
680  1000028  2621260       722945           buy  1511934727
682  1000028  1910706      1516409           buy  1512112339
733  1000028   756251      3158249          cart  1512314146


In [36]:
item_df_filtered['item_id'].shape

(5025,)

In [39]:
## getting the unique categories:

categories_df_filtered = taobao_filtered['category_id'].unique()
len(categories_df_filtered)

681

In [40]:
categories_df_filtered = pd.DataFrame({'category_id': categories_df_filtered})
categories_df_filtered.head()

,category_id
0,1029459
1,3108044
2,722945
3,1516409
4,3158249


In [42]:
categories_df_filtered = categories_df_filtered.sample(frac = 1, replace = False)

In [ ]:
categories_df_filtered.head()

,category_id
310,2217187
532,4640311
193,3277224
348,160181
90,1789614


In [44]:
print(categories_df_filtered.shape)

(681, 1)


In [ ]:
categories_df_filtered = categories_df_filtered.reset_index()
categories_df_filtered = categories_df_filtered.drop(columns = 'index')
categories_df_filtered = categories_df_filtered.reset_index()
categories_df_filtered = categories_df_filtered.rename(columns = {'index': 'new_categorymap_id'})


NameError: name 'categories_id_filtered' is not defined

In [46]:
categories_df_filtered.head()

,new_categorymap_id,category_id
0,0,2217187
1,1,4640311
2,2,3277224
3,3,160181
4,4,1789614


In [47]:
print(categories_df_filtered.shape)
print(len(categories_df_filtered['new_categorymap_id'].unique()))

(681, 2)
681


In [48]:
categories_df_filtered['new_categorymap_id'] = categories_df_filtered['new_categorymap_id'] + item_df_filtered['new_itemmap_id'].max() + 1
categories_df_filtered.head()

,new_categorymap_id,category_id
0,5025,2217187
1,5026,4640311
2,5027,3277224
3,5028,160181
4,5029,1789614


In [49]:
item_df_filtered['new_itemmap_id'].max()

5024

In [53]:
categories_df_filtered.to_csv('category_list_filtered2.csv', index = False)

In [54]:
trial_cat = pd.read_csv('category_list_filtered2.csv')
trial_cat.head()

,new_categorymap_id,category_id
0,5025,2217187
1,5026,4640311
2,5027,3277224
3,5028,160181
4,5029,1789614


In [59]:
item_df_filtered.head()

,new_itemmap_id,itemmap_id,item_id
0,0,4090,3840220
1,1,4490,5004166
2,2,5451,3245292
3,3,5571,3826932
4,4,6766,1358635


In [60]:
category_list2 = categories_df_filtered.copy(deep = True)
items_list2 = item_df_filtered[['new_itemmap_id', 'item_id']]

category_list2['type'] = 'category'
items_list2['type'] = 'item'

category_list2.rename(columns = {'new_categorymap_id':'new_map_id', 
                               'category_id': 'og_id'}, 
                    inplace = True)
items_list2.rename(columns = {'new_itemmap_id': 'new_map_id', 
                           'item_id': 'og_id'}, 
                inplace = True)

entity_filtered = pd.concat([items_list2, category_list2])

In [61]:
entity_filtered.head()

,new_map_id,og_id,type
0,0,3840220,item
1,1,5004166,item
2,2,3245292,item
3,3,3826932,item
4,4,1358635,item


In [62]:
len(entity_filtered['new_map_id'].unique())

5706

In [64]:
5025 + 681

5706

In [65]:
entity_filtered.to_csv('entity_list_filtered2.csv', index = False)

In [66]:
## loading in the user information : 
user_df_filtered = taobao_filtered['user_id'].unique()
len(user_df_filtered)

445661

In [67]:
user_df_filtered = pd.DataFrame({'user_id': user_df_filtered})
user_df_filtered.head()

,user_id
0,100
1,1000011
2,1000028
3,1000037
4,1000040


In [68]:
len(user_df_filtered)

445661

In [70]:
user_df_filtered = user_df_filtered.sample(frac = 0.25, replace =  False)

In [ ]:
user_df_filtered.head()
user_df_filtered = user_df_filtered.reset_index()
user_df_filtered = user_df_filtered.drop(columns = 'index')
user_df_filtered = user_df_filtered.reset_index()


,index,user_id
0,0,55835
1,1,492157
2,2,122151
3,3,687833
4,4,359258
...,...,...
111410,111410,800687
111411,111411,312281
111412,111412,841247
111413,111413,648406


In [74]:
user_df_filtered = user_df_filtered.rename(columns = {'index': 'new_usermap_id'})

In [75]:
user_df_filtered.head()

,new_usermap_id,user_id
0,0,55835
1,1,492157
2,2,122151
3,3,687833
4,4,359258


In [76]:
user_df_filtered.to_csv('user_list_filtered2.csv', index = False)

In [77]:
taobao_filtered2 = taobao_filtered[(taobao_filtered['user_id'].isin(user_df_filtered['user_id'])) &
                                   (taobao_filtered['item_id'].isin(item_df_filtered['item_id'])) &
                                   (taobao_filtered['category_id'].isin(categories_df_filtered['category_id']))]

taobao_filtered2.shape

(271821, 5)

In [79]:
taobao_filtered2.head()

,user_id,item_id,category_id,behavior_type,timestamp
126,100,4840649,1029459,buy,1511790130
680,1000028,2621260,722945,buy,1511934727
682,1000028,1910706,1516409,buy,1512112339
733,1000028,756251,3158249,cart,1512314146
934,1000040,3909833,570735,cart,1511755677


In [80]:
## creating the KG graph

taobao_filtered2 = taobao_filtered2[['item_id', 'category_id']]
taobao_filtered2 = taobao_filtered2.drop_duplicates()
taobao_filtered2.shape

(5030, 2)

In [85]:
trial = taobao_filtered2.groupby('item_id')['category_id'].nunique().reset_index()
trial[trial['category_id'] != 1]

,item_id,category_id
569,582354,2
598,615553,2
1902,1992652,2
2731,2794495,2
3026,3083057,2


In [87]:
## building the relations between the items and category ids
taobao_filtered3 = taobao_filtered2.merge(item_df_filtered, on = 'item_id', how = 'left')
print(taobao_filtered3.shape)
print(taobao_filtered3.head())

taobao_filtered3 = taobao_filtered3.merge(categories_df_filtered, on = 'category_id', how = 'left')
print(taobao_filtered3.head())
print(taobao_filtered3.shape)

(5030, 4)
   item_id  category_id  new_itemmap_id  itemmap_id
0  4840649      1029459            2777     2301214
1  2621260       722945            4264     3549836
2  1910706      1516409             370      290050
3   756251      3158249            1000      802511
4  3909833       570735            1598     1318233
   item_id  category_id  new_itemmap_id  itemmap_id  new_categorymap_id
0  4840649      1029459            2777     2301214                5071
1  2621260       722945            4264     3549836                5369
2  1910706      1516409             370      290050                5643
3   756251      3158249            1000      802511                5345
4  3909833       570735            1598     1318233                5219
(5030, 5)


In [88]:
taobao_kg_itemtocat = taobao_filtered3.copy(deep = True)
taobao_kg_cattoitem = taobao_filtered3.copy(deep = True)

taobao_kg_itemtocat = taobao_kg_itemtocat[['new_itemmap_id', 'new_categorymap_id']]
taobao_kg_cattoitem = taobao_kg_cattoitem[['new_categorymap_id', 'new_itemmap_id']]


taobao_kg_itemtocat['relation'] = 0
taobao_kg_cattoitem['relation'] = 1

taobao_kg_itemtocat = taobao_kg_itemtocat.rename(columns = {'new_itemmap_id': 'head', 
                                                              'new_categorymap_id': 'tail'})
taobao_kg_cattoitem = taobao_kg_cattoitem.rename(columns = {'new_categorymap_id': 'head', 
                                                            'new_itemmap_id': 'tail'})

print(taobao_kg_itemtocat.head())
print(taobao_kg_cattoitem.head())

   head  tail  relation
0  2777  5071         0
1  4264  5369         0
2   370  5643         0
3  1000  5345         0
4  1598  5219         0
   head  tail  relation
0  5071  2777         1
1  5369  4264         1
2  5643   370         1
3  5345  1000         1
4  5219  1598         1


In [89]:
kg_filtered = pd.concat([taobao_kg_itemtocat, taobao_kg_cattoitem])
kg_filtered = kg_filtered[['head', 'relation', 'tail']]

In [90]:
kg_filtered.head()

,head,relation,tail
0,2777,0,5071
1,4264,0,5369
2,370,0,5643
3,1000,0,5345
4,1598,0,5219


In [91]:
kg_filtered.shape

(10060, 3)

In [92]:
kg_filtered.to_csv('kg_filtered2.csv', index = False)

In [93]:
test_kg = pd.read_csv('kg_filtered2.csv')
test_kg.head()

,head,relation,tail
0,2777,0,5071
1,4264,0,5369
2,370,0,5643
3,1000,0,5345
4,1598,0,5219


In [ ]:
## creating te train and test sets
